In [1]:
# -*- coding: Paper1 -*-
"""
This code belongs to the paper:
-- An Iterative Gradient Descent-Based Reinforcement Learning Policy 
   for Active Control of Structural Vibrations, Computers & Structures.
-- Jagajyoti Panda, Mudit Chopra, Vasant Matsagar, Souvik Chakraborty, IIT Delhi.
   
This code is to present RL-based control algorithm in proportional (P) to state feedback.
-- Case study II: 8-story benchmark building
   State-space representation, Policy parameter update, Efficacy of trained RL-controller
"""
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
import os 
from numpy import abs as Abs
from numpy import sign as sign
from numpy import sqrt,pi,array
from scipy import signal
 
 
import matplotlib
import matplotlib.pyplot as plt

from control.matlab import *  # MATLAB-like functions
import control as ct
 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
# System dynamics for the MDOF system

# System parameters
nod = 1  
amp = 3  # amplification factor
n = 8    # Number of storey
r = 3    # no of MR dampers
theta = np.zeros((n,r))
theta[0,0]=1 
theta[3,1]=1
theta[5,2]=1

dampmax = 1000000*nod*amp    # Max MR Damper force
m = 345600   # Lumped mass in KG
M = np.diag(np.full(n,m))   
# print(M)    # Mass matrix
del m

kk = 3444*10**5
K = 2*np.diag(np.full(n,kk))
K[n-1,n-1] = kk
for x in range(n-1):
  K[x,x+1] = -kk
  K[x+1,x] = -kk
# print(K)   # stiffness matrix
del x,kk

cc = 2937000    # Inherent damping
C = 2*np.diag(np.full(n,cc))
C[n-1,n-1] = cc
for x in range(n-1):
  C[x,x+1] = -cc
  C[x+1,x] = -cc
# print(C)   # stiffness matrix
del x,cc

from numpy.linalg import eig
W,V=eig(np.linalg.inv(M) @ K)  # Eigen Analysis
omega=sqrt(W)  # Frequency rad/sec
oma=array(omega)  
fn=oma/(2*pi)  # Frequency Hz
# print('E-value:', omega)
# print('E-vector', V)
del W,V,omega,oma

gbar=np.diag(np.full(n,1))
for x in range(n-1):
  gbar[x,x+1] = -1
gamma=gbar @ theta
# print(gamma)

del x,theta

In [2]:
# Input Road Profile
T = 100
dt = 0.01
t = np.arange(0, T+dt, dt)
# amplitude   = 1*np.sin(t)
amplitude   = 1*np.sin(2 * np.pi * 0.927 * t)
amplitude1   = (1/2)*(np.sin(2 * np.pi * 0.927 * t) + np.sin(2 * np.pi * 2.749 * t))
amplitude2   = (1/3)*(np.sin(2 * np.pi * 0.927 * t) + np.sin(2 * np.pi * 2.749 * t) + np.sin(2 * np.pi * 4.478 * t))
len(t)
plt.figure()
plt.plot(t, amplitude,'r')
plt.figure()
plt.plot(t, amplitude1,'b')
plt.figure()
plt.plot(t, amplitude2,'g') 

In [ ]:
# Fast fourier transform (FFT)
sr = 1/dt  # sampling frequency
N1 = 2048  # Number of samples
fstep = sr/N1  # freq interval
f = np.linspace(0, (N1-1)*fstep, N1)

# Perform FFT
X = np.fft.fft(amplitude)
X_mag = np.abs(X)/N1

X1 = np.fft.fft(amplitude1)
X1_mag = np.abs(X1)/N1

X2 = np.fft.fft(amplitude2)
X2_mag = np.abs(X2)/N1

f_plot = f[0:int(N1/2+1)]/(2 * np.pi)
X_mag_plot = 2*X_mag[0:int(N1/2+1)]
X_mag_plot[0] = X_mag_plot[0] / 2 # DC component does not need to multiply by 2

X1_mag_plot = 2*X1_mag[0:int(N1/2+1)]
X1_mag_plot[0] = X1_mag_plot[0] / 2 # DC component does not need to multiply by 2

X2_mag_plot = 2*X2_mag[0:int(N1/2+1)]
X2_mag_plot[0] = X2_mag_plot[0] / 2 # DC component does not need to multiply by 2

# Plot 
fig, [ax1, ax2, ax3] = plt.subplots(nrows=3, ncols=1)
ax1.plot(f_plot, X_mag_plot, '.-')
ax2.plot(f_plot, X1_mag_plot, '.-')
ax3.plot(f_plot, X2_mag_plot, '.-')
ax3.set_xlabel('Freq (Hz)')
ax1.set_ylabel('FFT Amplitude |X(freq)|')
ax2.set_ylabel('FFT Amplitude |X(freq)|')
ax3.set_ylabel('FFT Amplitude |X(freq)|')
ax1.semilogx()
ax2.semilogx()
ax3.semilogx()
ax1.set_xlim(0, 100)
ax2.set_xlim(0, 100)
ax3.set_xlim(0, 100)
plt.show()

In [ ]:
# State space representation
I = -1*np.ones([n,1])  # Influence vector
A1 = np.zeros([n,n])
A2 = np.identity(n)
A3 = -1*np.linalg.inv(M) @ K
A4 = -1*np.linalg.inv(M) @ C
A = np.zeros([2*n,2*n])
A[0:n,n:2*n] = A2
A[n:2*n,0:n] = A3
A[n:2*n,n:2*n] = A4
# = np.ones((n,n), dtype=int)
print(A)  
B = np.zeros((2*n,r))
B[n:2*n,0:r] = np.linalg.inv(M) @ gamma   # Input matrix
print(B)
E = np.zeros((2*n,1))
E[n:2*n] = I   # Input matrix
print(E)
# Output matrix (Disp + Accel)
Cd = np.zeros([n+r, 2*n])
Cd[0:n, 0:n] = A3;
Cd[0:n, n:2*n] = A4;
Cd[n,0:2] =  np.array([[1, 1]])
Cd[n+1,2:5] =  np.array([[1, 1, 1]])
Cd[n+2,5:8] =  np.array([[1, 1, 1]])
print(Cd)
Dd = np.zeros([n+r, r])
Dd[0:n,0:r] = np.linalg.inv(M) @ gamma   # Input matrix
print(Dd)
Ed = np.zeros([n+r,1])
print(Ed)

In [ ]:
# Digital system (correct)
Ad =np.array([[0.9089,	0.0431,	0.0016,	0,	0,	0,	0,	0,	0.0089,	0.0005,	0,	0,	0,	0,	0,	0],
              [0.0431,	0.9104,	0.0432,	0.0016,	0,	0,	0,	0,	0.0005,	0.0089,	0.0005,	0,	0,	0,	0,	0],
              [0.0016,	0.0432,	0.9104,	0.0432,	0.0016,	0,	0,	0,	0,	0.0005,	0.0089,	0.0005,	0,	0,	0,	0],
              [0.0000,	0.0016,	0.0432,	0.9104,	0.0432,	0.0016,	0,	0,	0,	0,	0.0005,	0.0089,	0.0005,	0,	0,	0],
              [0.0000,	0,	0.0016,	0.0432,	0.9104,	0.0432,	0.0016,	0,	0,	0,	0,	0.0005,	0.0089,	0.0005,	0,	0],
              [0.0000,	0,	0,	0.0016,	0.0432,	0.9104,	0.0432,	0.0016,	0,	0,	0,	0,	0.0005,	0.0089,	0.0005,	0],
              [0.0000,	0,	0,	0,	0.0016,	0.0432,	0.9105,	0.0447,	0,	0,	0,	0,	0,	0.0005,	0.0089,	0.0005],
              [0.0000,	0,	0,	0,	0,	0.0016,	0.0447,	0.9536,	0,	0,	0,	0,	0,	0,	0.0005,	0.0094],
              [-17.2450,	7.8662,	0.4817,	0.0164,	0.0004,	0,	0,	0,	0.7618,	0.1102,	0.0057,	0.0002,	0,	0,	0,	0],
              [7.8662,	-16.7634,	7.8826,	0.482,	0.0164,	0.0004,	0,	0,	0.1102,	0.7675,	0.1104,	0.0057,	0.0002,	0,	0,	0],
              [0.4817,	7.8826,	-16.763,	7.8826,	0.482,	0.0164,	0.0004,	0,	0.0057,	0.1104,	0.7675,	0.1104,	0.0057,	0.0002,	0,	0],
              [0.0164,	0.482,	7.8826,	-16.763,	7.8826,	0.482,	0.0164,	0.0004,	0.0002,	0.0057,	0.1104,	0.7675,	0.1104,	0.0057,	0.0002,	0],
              [0.0004,	0.0164,	0.482,	7.8826,	-16.763,	7.8826,	0.4821,	0.0168,	0,	0.0002,	0.0057,	0.1104,	0.7675,	0.1104,	0.0057,	0.0002],
              [0.0000,	0.0004,	0.0164,	0.482,	7.8826,	-16.763,	7.883,	0.4985,	0,	0,	0.0002,	0.0057,	0.1104,	0.7675,	0.1104,	0.0059],
              [0.0000,	0,	0.0004,	0.0164,	0.4821,	7.883,	-16.7466,	8.3647,	0,	0,	0,	0.0002,	0.0057,	0.1104,	0.7677,	0.1161],
              [0.0000,	0,	0,	0.0004,	0.0168,	0.4985,	8.3647,	-8.8804,	0,	0,	0,	0,	0.0002,	0.0059,	0.1161,	0.8779]])
print(Ad)
Bd  = np.array([[1.34702468304747E-10,	-1.15672784046959E-13,	-3.33223374028602E-17],
               [4.80538564665434E-12,	-4.68967910640923E-12,	-2.18046675626884E-15],
               [1.17886577963029E-13,	-1.30012788764478E-10,	-1.15706111159729E-13],
               [2.2137939169218E-15,	1.30012788769298E-10,	-4.68967953544642E-12],
               [3.37561986981929E-17,	4.68967953549301E-12,	-1.30012788764477E-10],
               [4.33862421178217E-19,	1.15706111207136E-13,	1.30012789198338E-10],
               [4.82365420392012E-21,	2.1804715802934E-15,	4.68971286260631E-12],
               [4.76906938178176E-23,	3.37561991045439E-17,	1.17886577963177E-13],
               [2.57851128124862E-08,	-4.88918641729282E-11,	-2.13266602760077E-14],
               [1.49750412070413E-09,	-1.4485906057431E-09,	-1.1612850802489E-12],
               [5.00744800217048E-11,	-2.43365218826153E-08,	-4.89131948958366E-11],
               [1.182615848769E-12,	2.43365218867235E-08,	-1.44859092576268E-09],
               [2.16507881080578E-14,	1.44859092580829E-09,	-2.43365218826153E-08],
               [3.2412782161542E-16,	4.89131949414518E-11,	2.43365222067431E-08],
               [4.1082484210822E-18,	1.16128918848908E-12,	1.4486122565312E-09],
               [4.56140126500856E-20,	2.16507881049024E-14,	5.0074480021704E-11]])
print(Bd)

Ed = np.array([[-4.82554328318935E-05,	-4.99576926176694E-05,	-4.99992111239577E-05,	-4.99999880289061E-05,	-4.99999998466733E-05,	-4.99999999982996E-05,	-4.99999999999823E-05,	-4.99999999999999E-05,	-0.00944659446040994,	-0.00998185393282461,	-0.009999575981124,	-0.00999999228912783,	-0.00999999988509437,	-0.0099999999985485,	-0.00999999999998409,	-0.00999999999999984,	]]).T
print(Ed)
np.save('Ad.npy', Ad)
np.save('Bd.npy', Bd)
np.save('Ed.npy', Ed)

ko = matrix_rank(ctrb(Ad, Bd))
if ko == len(Ad):
  print("System is controllable")
else:
  print("System is uncontrollable")

In [ ]:
# Initial controller gain (proportional)
Qp_i = np.identity(2*n)
# Kp, Sp, ep = ct.dlqr(Ad, Bd, Qp_i, Rp)
# print(Kp)  # Discrete time
(i,j)=Bd.shape
Rp_i =  (10**(-10))*np.identity(j)
Rp_o =  (10**(-12.1))*np.identity(j)
Kp, Sp, ep = ct.dlqr(Ad, Bd, Qp_i, Rp_i)
Kpo, Spo, epo = ct.dlqr(Ad, Bd, Qp_i, Rp_o)
print(Kp)
print(Kpo)
# Time series analysis
xo = np.zeros((len(t),2*n))
(i,j) = Cd.shape
yo = np.zeros((len(t),i))
xp = np.zeros((len(t),2*n))
xp2 = np.zeros((len(t),2*n))
(i,j) = Cd.shape
yp = np.zeros((len(t),i))
yp2 = np.zeros((len(t),i))
del i,j

for tkk in range(0,len(t)-1):
    time_current = t[tkk]
    y = (Cd @ np.array([xo[tkk,:]]).T)
    yo[tkk,:] = y.T
    x = (Ad @ np.array([xo[tkk,:]]).T) + (Ed * amplitude[tkk])
    xo[tkk+1,:] = x.T
    y1 = (Cd @ np.array([xp2[tkk,:]]).T) + (Dd @ ((-1*Kpo) @ np.array([xp2[tkk,:]]).T))
    yp2[tkk,:] = y1.T
    x1 = (Ad @ np.array([xp2[tkk,:]]).T) + (Bd @ ((-1*Kpo) @ np.array([xp2[tkk,:]]).T)) + (Ed * amplitude[tkk])
    xp2[tkk+1,:] = x1.T
    y2 = (Cd @ np.array([xp[tkk,:]]).T) + (Dd @ ((-1*Kp) @ np.array([xp[tkk,:]]).T))
    yp[tkk,:] = y2.T
    x2 = (Ad @ np.array([xp[tkk,:]]).T) + (Bd @ ((-1*Kp) @ np.array([xp[tkk,:]]).T)) + (Ed * amplitude[tkk])
    xp[tkk+1,:] = x2.T
for i in range(2*n):
    plt.figure()
    plt.plot(t, xo[:,i], 'r', t, xp[:,i], 'b', t, xp2[:,i], 'g')

In [ ]:
# Bode plot for the system (uncontrolled)
BKt = np.load('BKt_P8.npy')
Cd1 = np.load('Cd1_P8.npy')
plt.figure()
bode(ss(Ad, Ed, Cd[1, :], np.zeros([1]), dt), logspace(-2, 2), plot=True)
bode(ss(Ad-(Bd@Kpo), Ed, Cd[1, :]-(Dd[1, :]@Kpo), np.zeros([1]), dt), logspace(-2, 2), plot=True)
mag_pc, phase_pc, om_pc = bode(ss(Ad-BKt, Ed, Cd1[1, :]-BKt[1,:], np.zeros([1]), dt), logspace(-2, 2), plot=True)

In [32]:
# propertional (Discrete)
Bj_p = Bd @ (-1*Kp) 
Al_p = Ad.T + Bj_p.T 
Qp = np.identity(2*n)
P_p = dlyap(Al_p, -Qp)
Ki = P_p@Bj_p

In [ ]:
# P-controller (discrete)
# Policy parameter update
Zmax = 501 # No of episodes
# Qp = np.array([[Ks, -Ks, 0, 0],
#             [-Ks, Ks+Kt, 0, 0],
#             [0, 0, M, 0],
#             [0, 0, 0, m]])
Qp = np.identity(2*n)
BK = np.zeros([2*n,2*n,Zmax])
BK[0:2*n,0:2*n,0] = Bj_p  # Initial Stiffness
djdk_d = np.zeros([2*n,2*n,Zmax])  # Gradient
alpha = 40.0  
xd = np.zeros([len(t), 2*n, Zmax])  # State
pd = np.zeros([len(t), 2*n, Zmax])  # Adjoint State
done = False
J_cd = np.zeros([len(t), 1, Zmax])  # Cost function
E_cd = np.zeros([len(t), 1, Zmax])  # Structure's energy
val_arr = np.zeros([len(t), 1, Zmax]) # x.T*K*x
sd = []  # number of tau_s

for id in range(0, Zmax-1):  
  BKo = BK[0:2*n,0:2*n,id] 
  print(BKo)
  for tkk in range(0,len(t)-1):
    time_current = t[tkk]
    h = amplitude[tkk] 
    xn = np.array([xd[tkk,:,id]]).T
    xn1 = (Ad @ xn) + (BKo @ xn) + (Ed * h)
#     print(xn1)
    xd[tkk+1,:,id] = xn1.T
    En1 = 0.5*((xn1.T @ Qp)@xn1)
    E_cd[tkk+1,:,id] = En1
    Jn1 = J_cd[tkk,:,id] + dt*(En1)
    J_cd[tkk+1,:,id] = Jn1  # Cost functional
    del xn,xn1
  print(Jn1)  
  # Adjoint State  
  sd.append([])
  for tkk in range(0,len(t)-1):
    i = len(t)-1-tkk
#     print(i)
    xn = np.array([xd[i,:,id]]).T
    val = (xn.T @ BKo) @ xn
#     print(val)
    val_arr[i,:,id] = val
    if(val==0):
      pn = np.array([pd[i,:,id]]).T
      sd[id].append(i)
      d_ptau1 = (pn.T@BKo)@xn  # (1*n)@(n*n)@(n*1)
#       print(val)
      d_ptau = (d_ptau1[0,0]*((BKo + BKo.T)@xn))      #1*(1*1)*((n*n)@(n*1))  #############
#       d_ptau = (d_ptau1[0,0]*xn)      #1*(1*1)*((n*n)@(n*1))  #############
#       print(d_ptau)
      pnb1 = pn + d_ptau  # (n*1)
      pd[i-1,:,id] = pnb1.T
    else:
      if(val[0]<0.000001 and val[0] >-0.000001):
        sd[id].append(i)
      pn = np.array([pd[i,:,id]]).T
      F = Qp@xn 
      pnb1 = (-Ad.T @ pn) + (-BKo.T @ pn) + (F)
      pd[i-1,:,id] = pnb1.T
#     print(p[i-1,:,id])
    del xn, val, pn, pnb1, i
    
  # Evaluate dj/dk
  sum = djdk_d[0:2*n,0:2*n,id]
  print(len(sd[id]))
  if(len(sd[id])>0):
    for j in sd[id]:
        xn = np.array([xd[j,:,id]]).T
        pn = np.array([pd[j,:,id]]).T
        sum1 = (pn.T@BKo)@xn  # (1*n)@(n*n)@(n*1)
        sum = sum +  ((-1)*(sum1[0,0]*(xn@xn.T)))  # 1*(1*1)*((n*1)@(1*n))  #############
    print(sum)
    djdk_d[0:2*n,0:2*n,id] = sum
  # Update K 
#     alpha = 2*id
    BK[0:2*n,0:2*n,id+1] = BK[0:2*n,0:2*n,id]  - alpha*djdk_d[0:2*n,0:2*n,id]
  else:
    print(np.linalg.norm(djdk_d[0:2*n,0:2*n,id]))
#     break
      

In [ ]:
# Figure (dj_dk, cost  functional)
z = np.arange(0, Zmax-1, 1)
n_djdk = np.zeros([Zmax-1, r])
J_Tl = np.zeros([Zmax-1, r])
del id
for id in range(0, Zmax-1):
    norm_djdk = np.linalg.norm(djdk_d[0:2*n,0:2*n,id])
    n_djdk[id,:] = norm_djdk
    J_T = J_cd[len(t)-1,:,id]
    J_Tl[id, :] = J_T
plt.figure()
plt.plot(z, n_djdk)
plt.figure()
plt.plot(z, J_Tl)

In [ ]:
# Maximum value of response
for i in range(0,n):
    print("Peak Dis of floor:", i+1, "Uncontrolled:", max(xo[:,i], key=abs), "Optimal-P:", max(xp2[:,i], key=abs), "PG-P:", max(xd[:,i,Zmax-2], key=abs))
    print("Peak Vel of floor:", i+1, "Uncontrolled:", max(xo[:,n+i], key=abs), "Optimal-P:", max(xp2[:,n+i], key=abs), "PG-P:", max(xd[:,n+i,Zmax-2], key=abs))
    print("Peak Acc of floor:", i+1, "Uncontrolled:", max(yo[:,i], key=abs), "Optimal-P:", max(yp2[:,i], key=abs), "PG-P:", max(yp_zmax.T[:,i], key=abs))